This file reads project information from Devrel airtable and sends to Dune (note: private to labs table)

This code is most similar to [op-analytics/op_chains_tracking/read_op_chain_metadata_airtable.ipynb](https://github.com/ethereum-optimism/op-analytics/blob/main/op_chains_tracking/read_op_chain_metadata_airtable.ipynb)

In [ ]:
import pandas as pd

import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import airtable_utils as a
import pandas_utils as p
sys.path.pop()

import dotenv
import os
dotenv.load_dotenv()

at_base_id = os.environ["AIRTABLE_DEVREL_BASE_ID"]

columns = ['Team Name','Priority','Owner..name','Partner Type','Partner Category','Lead Source']

In [ ]:
filter = \
"""
IF(
  AND(
    OR(
      Priority = 'P0',
      Priority = 'P1',
      Priority = 'P2',
      Priority = 'Superchain'
    ),
    NOT(
      BLANK({Partner Type})
    )
  ),
  1,0
)
"""

df = a.get_dataframe_from_airtable_database(at_base_id,'Teams', filter)

df = df[columns]

df = df.rename(columns={'Owner..name':'Owner'})
print(len(df))

In [ ]:
# Filter the DataFrame
filtered_df = df[df['Partner Type'].notnull() \
                 & df['Priority'].isin(['P0','P1','P2','Superchain'])]
filtered_df.reset_index(inplace=True, drop=True)



# Format timeseries columns
# filtered_df = p.format_datetime_columns(filtered_df, format='%Y-%m-%dT%H:%M:%S.%fZ')
display(filtered_df)

In [ ]:
# Upload to Dune

# Post to Dune API
table_name = os.environ["DUNE_BD_AT_TBL_NAME"]
d.write_dune_api_from_pandas(filtered_df, table_name,\
                             '')